<h3> <strong> 1. 실험 </strong> </h3>

먼저 아래 링크를 참고해 yolov5x6.yaml를 만들고 학습 명령을 수행시켜야 한다.

* http://khuhub.khu.ac.kr/2021-1-capstone-design1/HCS_Project2/blob/ef6fbb8b5035846aad6a0a45c35657a6091b98cc/YOLOv5/models/hub/yolov5x6.yaml

* 학습 명령: <br>
(1) python train.py --img 1024 --batch 8 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x6.yaml' --weights yolov5x6.pt --hyp './runs/evolve/YoloV5x_Hyp_tuning/hyp_evolve.yaml' --label-smoothing 0.1 --name YoloV5x6_base --cache <br>
(2) python train.py --img 1024 --batch 4 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x6.yaml' --weights yolov5x6.pt --hyp './runs/evolve/YoloV5x_Hyp_tuning/hyp_evolve.yaml' --label-smoothing 0.1 --name YoloV5x6_multi_scale --cache --multi-scale

* 테스트 명령: <br>
(1) python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x6_augmentation/weights/best.pt' --save-txt --save-conf --name YoloV5x6_base --augment <br>
(2) python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x6_multi_scale/weights/best.pt' --save-txt --save-conf --name YoloV5x6_multi_scale --augment


In [1]:
import pandas as pd
import os

df = pd.DataFrame(index=[i for i in range(4871)], columns=['PredictionString', 'image_id'])
yolo_path = './../yolov5/runs/detect/YoloV5x6_multi/labels'
for img_index in range(4871):
    image_id = str(img_index)
    while len(image_id)<4: image_id = '0'+image_id

    PredictionString = ''
    try: 
        txt_file = open(os.path.join(yolo_path, image_id+'.txt'), 'r')
        lines = txt_file.readlines()
    
        for line in lines:
            category, yolo_x, yolo_y, yolo_w, yolo_h, pred = (line.split())
            x_min = (float(yolo_x)-float(yolo_w)/2)*1024
            y_min = (float(yolo_y)-float(yolo_h)/2)*1024
            x_max = x_min+float(yolo_w)*1024
            y_max = y_min+float(yolo_h)*1024
            PredictionString += f'{category} {pred} {x_min} {y_min} {x_max} {y_max} '
        txt_file.close()
    except FileNotFoundError:
        pass
    
    df.iloc[img_index] = [PredictionString, 'test/'+image_id+'.jpg']

df.to_csv('./YoloV5x6_multi.csv', index=False)

<h3> <strong> 2. 결과 </strong> </h3>

(1) yolov5x6 모델 적용: 0.5587 / yolov5x(0.5017)에 비해 높은 성적을 기록함. <br>
(2) multi-scale과 label smoothing 동시에 적용시 오히려 성능이 감소함. valid셋에 대한 mAP 성적이 낮아서 제출은 하지 않음.